<a href="https://colab.research.google.com/github/Pavun-KumarCH/Medical-Inventory-Management-Machine-Learning/blob/main/Medical_Inventory_Management_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<h1><center><strong><font size="6">Medical Inventory Management Project<font><strong></center></h1>

## **CRISP - ML(Q)**

**CRISP-ML(Q) process model describes six phases:**

- Business and Data Understanding
- Data Preparation (Data Engineering)
- Model Building (Machine Learning)
- Model Evaluation and Tunning
- Deployment
- Monitoring and Maintenance

---

## **Problem Statements**:
Bounce rate is increasing significantly leading to patient dissatisfaction.

---

### **Business Objective** :
Minimize Bounce Rate.

### **Business Constraints** :
Minimize Inventory Cost.

---
          
## **Success Criteria** : -

**Business Success Criteria** : Reduce bounce rate by at least 30%

**Machine Learning Success Criteria** : Achieve an Accuracy of at least 90%

**Economic Success Criteria** : Increase revenue by at least 20 lacs INR by reducing bounce rate.

---

### **Data Collection** :
                  
Data Was Provided client which One of the Leading Pharma Company in india.


## **Data Description** :
The dataset consists of 14218 entries with the following columns:

**VARIABLE NAME - DESCRIPTION**

---
1. **Typeofsales** :	*Type of sale of the drug. Either the drug is sold or returned.*

2. **Patient_ID** : 	*ID of a patient*

3. **Specialisation** :	*Name of Specialisation (eg. Cardiology)*

4. **Dept** :	        *Pharmacy, the formulation is related with.*

5. **Dateofbill** :  	*Date of purchase of medicine*

6. **Quantity** :	    *Quantity of the drug*

7. **ReturnQuantity** :	*Quantity of drug returned by patient to the pharmacy*

8. **Final_Cost** :	    *Final Cost of the drug (Quantity included)*

9. **Final_Sales** :	*Final sales of drug*

10. **RtnMRP** :	       * MRP of returned drug (Quantity included)*

11. **Formulation** :	*Type of formulation*

12. **DrugName** :	    *Generic name of the drug*

13. **SubCat** :	        *Subcategory (Type) to the category of drugs*

14. **SubCat1** :     	*Subcategory (condition) to the category of drugs*


In [ ]:
# Import required libraries
import io
import pylab
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from kneed import KneeLocator

import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
import tslearn
from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw

import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
import joblib


In [ ]:
# Suppress Warning
import warnings
warnings.filterwarnings('ignore')

### Load The Data

In [ ]:
# from google.colab import files
# uploaded = files.upload()
# filename = next(iter(uploaded))
# filename

In [ ]:
# medical_inv = pd.read_excel(io.BytesIO(uploaded[filename]))


In [ ]:
medical_inv = pd.read_excel('/Users/pavankumar/Projects/Medical Inventory Management/Datasets/Medical Inventory Optimaization Dataset.xlsx')


### Retrive the Data Info

In [ ]:
medical_inv.info()

In [ ]:
medical_inv

# **Financial Metric's analysis**

Based on the columns 'Quantity', 'Final_Cost', 'Final_Sales', and 'RtnMRP', we can calculate various metrics that provide insights into sales data. Here are some additional calculations we perform:


---


##Total Revenue:
    This is the same as Final_Sales.


---


##Total Cost:
    This is the same as Final_Cost.


---


##Total Profit:
    Calculated as Final_Sales - Final_Cost.


---


##Profit Margin:

    Calculated as (Final_Sales - Final_Cost) / Final_Sales. This represents the percentage of revenue that turns into profit.


---


##Return on Investment (ROI):

    Calculated as (Final_Sales - Final_Cost) / Final_Cost. This measures the profitability of the investment.


---


##Average Selling Price (ASP):

    Calculated as Final_Sales / Quantity. This represents the average price at which each unit is sold.


---


##Average Cost Price (ACP):

    Calculated as Final_Cost / Quantity. This represents the average cost of each unit.


---


##Markup:

    Calculated as (Final_Sales - Final_Cost) / Final_Cost. This represents the percentage increase over the cost price.


---


##Return Margin:

    Calculated as (RtnMRP - Final_Cost) / RtnMRP. This indicates the margin when considering the maximum retail price (MRP) for returns.


---


##Revenue per Quantity:

    Calculated as Final_Sales / Quantity. This is similar to ASP but ensures clarity.


In [ ]:
# Create a DataFrame to store all related Sales Metrics
sales_df = pd.DataFrame(columns = ['Profit'])

# Calculate Total Profit
sales_df['Profit'] = pd.DataFrame(medical_inv['Final_Sales'] - medical_inv['Final_Cost'])

# Calculate Profit Margin
sales_df['Profit Margin'] = sales_df['Profit'] / medical_inv['Final_Sales']

# Calculate Return on Investment(ROI)
sales_df['ROI'] = (medical_inv['Final_Sales'] - medical_inv['Final_Cost'])/ medical_inv['Final_Cost']

# Calculate Average Selling Price(ASP)
sales_df['ASP'] = medical_inv['Final_Sales']/ medical_inv['Quantity']

# Calculate Avreage Cost Price(ACP)
sales_df['ACP'] = medical_inv['Final_Cost']/ medical_inv['Quantity']

# Calculate Profit per Unit
sales_df['Profit per Unit'] = sales_df['Profit'] / medical_inv['Quantity']

# Calculate Markup
sales_df['Mark up'] = sales_df['Profit'] / medical_inv['Final_Cost']

# Calculate Return Mark up
sales_df['Return Margin'] = (medical_inv['RtnMRP'] - medical_inv['Final_Cost'])/  medical_inv['RtnMRP']


print(f"The Total Profit sales: {sales_df['Profit'].sum()}\n")

round(sales_df.describe())

In [ ]:
sales_df.head()

# **Data Preparation**

## Duplicates Handling


In [ ]:
duplicates = medical_inv.duplicated()
sum(duplicates)

In [ ]:
medical_inv.drop_duplicates(inplace = True)
duplicates = medical_inv.duplicated()
sum(duplicates)

## Handling Missing Values

In [ ]:
medical_inv.isna().sum()


### **Imputation**

In [ ]:
# Reset the index before applying group-wise mode
medical_inv.reset_index(drop = True, inplace = True)

In [ ]:
# Impute missing values in Formulation column based on the mode of the group
group_cols = ['Typeofsales','Specialisation','Dept']

for col in ['Formulation', 'DrugName', 'SubCat', 'SubCat1']:
    medical_inv[col] = medical_inv.groupby(group_cols)[col].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

medical_inv.isna().sum()

In [ ]:
# We still have few missing values
medical_inv.dropna(inplace = True)
medical_inv.reset_index(drop = True, inplace = True)
medical_inv.isna().sum()

## Data Manupulation

In [ ]:
# @title Date Manipulation
medical_inv['Dateofbill'] = pd.to_datetime(medical_inv['Dateofbill'])

# Sort the datadet based on date column in ascending order
medical_inv = medical_inv.sort_values(by = 'Dateofbill', ascending = True)

In [ ]:
# @title Date Range
start_date = medical_inv['Dateofbill'].min()
end_date = medical_inv['Dateofbill'].max()

print(f'Date Range: [{start_date}, {end_date}]')

In [ ]:
# @title Converstion of Date to Months and Weeks
# Converting date format to month
medical_inv['Months'] = medical_inv['Dateofbill'].dt.strftime("%b")

# Converting date format to Week
medical_inv['Week'] = medical_inv['Dateofbill'].dt.isocalendar().week

# Here in isocalendar a week starts from 1st thrusday so same days in jan might show as week 52 so we are filtering it
medical_inv.loc[(medical_inv['Week'] == 52) & (medical_inv['Dateofbill'].dt.month == 1), 'Week'] = 1

medical_inv.reset_index(drop = True, inplace = True)

In [ ]:
display(medical_inv.head(5))

In [ ]:
display(medical_inv.tail(5))

In [ ]:
# Speifying columns Final cost and final sale  to round
medical_inv['Final_Cost'] = medical_inv['Final_Cost'].map(lambda x : round(x))
medical_inv['Final_Sales'] = medical_inv['Final_Sales'].map(lambda x : round(x))

## Feature Selection

In [ ]:
# droping Irrelavent columns
medical_inv.drop(['Patient_ID','ReturnQuantity'], axis = True, inplace = True)

In [ ]:
medical_inv.head(10)

# **Descriptive Analytics**

In [ ]:
round(medical_inv.describe())

## Segregate Numeric and Non numeric columns


In [ ]:
# Numerical Features
numeric_features = medical_inv.select_dtypes(exclude = ['object','datetime64']).columns
numeric_features = numeric_features.drop('Week')
display(numeric_features)


# Categorical Features
categorical_features = medical_inv.select_dtypes(include = ['object']).columns
display(categorical_features)

## First Moment Decision - Measure Of Central Tendency

We see that "MULTIPLE ELECTROLYTES 500ML IVF"	 is the top performing drug

In [ ]:
# Mean
medical_inv[numeric_features].mean()

In [ ]:
# Median
medical_inv[numeric_features].median()

In [ ]:
# Mode
medical_inv.mode()

## Second Moment Bussiness Decision - Measure of Dispersion

In [ ]:
# Variance
medical_inv[numeric_features].var()

In [ ]:
# Standard Deviation
medical_inv[numeric_features].std()

## Third Moment Business Decision - Skewness

In [ ]:
# Skewness
medical_inv[numeric_features].skew()

## Forth Moment Business Decision - Kurtosis

In [ ]:
# Kurtosis
medical_inv[numeric_features].kurt()

# **Exploratory Data Analysis**

## Data Quantity Distribustion

In [ ]:
# colors to cycle through
colors = ['blue', 'red', 'green', 'orange']


def EDA(column, colors=colors):
    # Get the maximum value of the column
    max_value = medical_inv[column].max()

    # Randomly select a color from the list
    color = np.random.choice(colors)

    # Create the histogram
    plt.hist(medical_inv[column], color=color, bins=20, alpha=0.7)

    # Set the title and labels
    plt.title(f'Data Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

    # Set the x-axis limit
    plt.xlim(0, max_value)
    plt.show()


In [ ]:
columns = list(numeric_features)

for column in columns:
  EDA(column)


In [ ]:
stats.probplot(medical_inv.Quantity, dist = 'norm', plot = pylab)
plt.show()

## Log Transformation

In [ ]:
# Transforming the data to a normal distribution
stats.probplot(np.log(medical_inv.Quantity), dist = 'norm', plot = pylab)
plt.show()


## Barplot Quantity of Drug sold by Month


In [ ]:
sns.barplot(data = medical_inv, x = 'Months', y = 'Quantity', palette='muted')
plt.title('Quantity of Drugs sold by month')
plt.show()



---



---



# **Top Performing drugs**

In [ ]:
top_perform_quantity = medical_inv.groupby(['DrugName','SubCat','SubCat1'], as_index = False)['Quantity'].sum()
top_perform_quantity.sort_values(by = 'Quantity',ascending= False,inplace = True)
display(top_perform_quantity.head(20))



---





In [ ]:
# @title Based on Quantity
top_drugs = medical_inv.groupby(['DrugName'], as_index = False)['Quantity'].sum()
top_drugs.sort_values(by = 'Quantity',ascending= False,inplace = True)

top_num_drugs = top_drugs.head(50)
display(top_num_drugs)

In [ ]:
monthly = pd.DataFrame(medical_inv.groupby('Months')['Quantity'].sum())
# Create Dictionary to map month names into numeric values
dict_month = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
              'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

# Map month names to numeric values and sort the DataFrame
monthly['MonthNum'] = monthly.index.map(dict_month)
monthly = monthly.sort_values('MonthNum')
monthly.drop(['MonthNum'], axis=1, inplace=True)

# Display the result
display(monthly)

In [ ]:
# @title Monthly Quantity
Monthly_data = medical_inv.groupby(['DrugName', 'Months'])['Quantity'].sum().reset_index()
# Create Dictionary to map month names into numeric values
dict_month = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
              'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
# Map month names to numeric values and add a new column 'MonthNum'
Monthly_data['MonthNum'] = Monthly_data['Months'].map(dict_month)
# Sort the DataFrame by 'MonthNum'
Monthly_data = Monthly_data.sort_values(by='MonthNum')
# Drop the 'Months' column if it's no longer needed
Monthly_data.drop(['MonthNum'], axis=1, inplace=True)
# Display the result
display(Monthly_data)

# **Filtered Data**

In [ ]:
#Filter the original dataset to include only the top 20 drugs
top_drug_names = top_num_drugs['DrugName'].tolist()
filtered_data = Monthly_data[Monthly_data['DrugName'].isin(top_drug_names)]
# Display the first few rows of the filtered dataset
filtered_data.to_csv('Medical_inv_Data.csv')
filtered_data

In [ ]:
# Create Dictionary to map month names into numeric values
dict_month = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
              'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

# Pivot the DataFrame Based on DrugName
pivoted_table = filtered_data.pivot_table(index='Months', columns='DrugName', values='Quantity', aggfunc='sum')

# Fill NaN values with 0
pivoted_table = pivoted_table.fillna(0)

# Map month names to numeric values and sort the DataFrame
pivoted_table['MonthNum'] = pivoted_table.index.map(dict_month)
pivoted_table = pivoted_table.sort_values('MonthNum')
pivoted_table.drop(['MonthNum'], axis=1, inplace=True)

# Display the result
pivoted_table.head(52)

In [ ]:
## Ploting For Each Column
for column in pivoted_table.columns:
  plt.figure(figsize = (15,5))
  plt.plot(pivoted_table.index, pivoted_table[column], color = 'orange', marker = '*', linestyle = '-', label = column)
  plt.title(f'Time Series of {column}')
  plt.xlabel('Month of the Year')
  plt.xticks(range(len(pivoted_table.index)), pivoted_table.index, rotation=45)
  plt.ylabel(column)
  plt.legend()
  plt.tight_layout()
  plt.grid()
  plt.show()

# **Clustering**


In [ ]:
# Transpose the table to have drugs as rows and weeks as columns
pivoted_table_T = pivoted_table.T

# Normalize the data
scaler = MaxAbsScaler()
normalized_data = scaler.fit_transform(pivoted_table_T)

# Choose the number of clusters (adjust as needed)
n_clusters = 3

# Apply K-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
drug_clusters = kmeans.fit_predict(normalized_data)

# Add cluster labels to the DataFrame
pivoted_table_T['Cluster'] = drug_clusters

# Analyze the clusters
cluster_summary = pivoted_table_T.groupby('Cluster').mean()
display(cluster_summary)

# Compute the silhouette score
silhouette_avg = silhouette_score(normalized_data, drug_clusters)
display(f'Silhouette Score: {silhouette_avg:.2f}')


# Determine the optimal number of clusters using the elbow method
TWSS = []
k_range = list(range(1, 12))
for k in k_range:
  kmeans = KMeans(n_clusters = k)
  kmeans.fit(normalized_data)
  TWSS.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_range, TWSS, 'ro-')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method For Optimal k')
plt.show()

from kneed import KneeLocator
kl = KneeLocator(k_range, TWSS, curve = 'convex', direction = 'decreasing', interp_method = "interp1d" )
kl.elbow
plt.plot(k_range, TWSS)
plt.xticks(k_range)
plt.ylabel("Inertia")
plt.axvline(x = kl.elbow, color = 'r',label = 'axvline', ls = '--')
plt.legend()
plt.show()

# Visualization for clustering using PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(normalized_data)

plt.figure(figsize=(10, 6))
plt.scatter(principal_components[:, 0], principal_components[:, 1], c=drug_clusters, cmap='viridis')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA of Drug Clusters')
plt.tight_layout()
plt.colorbar(label='Cluster')
plt.grid()
plt.show()

# Compute the silhouette score
silhouette_avg = silhouette_score(normalized_data, drug_clusters)
print(f'Silhouette Score: {silhouette_avg:.2f}')


In [ ]:
# @title **Kmeans Time Series Clustering**
# Calculate features
features = pivoted_table_T.apply([np.mean, np.std, np.max, np.min], axis=1)
features['trend'] = pivoted_table_T.apply(lambda x: np.polyfit(range(len(x)), x, 1)[0], axis=1)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Reshape the data for TimeSeriesKMeans (tslearn requires a specific shape)
features_reshaped = features_scaled.reshape((features_scaled.shape[0], features_scaled.shape[1], 1))

# Initialize and fit the TimeSeriesKMeans model with the optimal number of clusters
optimal_clusters = 3 # Replace this with the actual optimal k from the elbow curve
model = TimeSeriesKMeans(n_clusters=optimal_clusters, metric="dtw", random_state=42)
clusters = model.fit_predict(features_reshaped)

# Add cluster labels to the DataFrame
pivoted_table_T['Cluster'] = clusters

# Calculate the Silhouette Score
sil_score = silhouette_score(features_scaled, clusters)
display(f'Silhouette Score for {optimal_clusters} clusters: {sil_score}')# Plot the Silhouette Score for different k values
sil_scores = []
for k in k_range[1:]:
    model = TimeSeriesKMeans(n_clusters=k, metric="dtw", random_state=42)
    labels = model.fit_predict(features_reshaped)
    sil_score = silhouette_score(features_scaled, labels)
    sil_scores.append(sil_score)

plt.figure(figsize=(10, 6))
plt.plot(k_range[1:], sil_scores, marker='o', color = 'g')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs. Number of Clusters')
plt.show()

## Cluster Evaluation

In [ ]:
from sklearn import metrics

round(metrics.silhouette_score(features_scaled, clusters),2)

In [ ]:
# @title Sorting the data By Clusters
# Sorting the data by 'Cluster' column
sorted_data = pivoted_table_T.sort_values(by='Cluster')

clusters = sorted_data

clusters = clusters.T

In [ ]:
# Function to plot sales data for each drug within a cluster
def plot_drug_sales(cluster_data, cluster_number):
    drugs = cluster_data.columns
    for drug in drugs:
        drug_data = cluster_data[drug].dropna()
        if not drug_data.empty:
            plt.figure(figsize=(10, 6))
            plt.plot(drug_data.index, drug_data.values, marker='o')
            plt.xlabel('Date')
            plt.ylabel('Quantity Sold')
            plt.title(f'{drug} Sales in Cluster {cluster_number}')
            plt.xticks(rotation=45)
            plt.grid(True)
            plt.show()

# Create separate DataFrames for each cluster
cluster_0 = sorted_data[sorted_data['Cluster'] == 0].drop(columns='Cluster').T
cluster_1 = sorted_data[sorted_data['Cluster'] == 1].drop(columns='Cluster').T
cluster_2 = sorted_data[sorted_data['Cluster'] == 2].drop(columns='Cluster').T

In [ ]:
# @title Drugs in Each Cluster
# Create a mapping between drug names and clusters
drug_cluster_mapping = {}

for cluster_num, cluster_data in zip(range(n_clusters), [cluster_0, cluster_1, cluster_2]):
    for drug_name in cluster_data.columns:
        drug_cluster_mapping[drug_name] = cluster_num

# Add the 'Cluster' column to the filtered_data DataFrame
filtered_data['Cluster'] = filtered_data['DrugName'].map(drug_cluster_mapping)
grouped_data = filtered_data.groupby('Cluster')

for cluster, group_data in grouped_data:
    unique_drug_names = group_data['DrugName'].unique()
    unique_drug_count = len(unique_drug_names)
    print(f"Cluster {cluster}:,\n")
    print(f"Unique Drug Names ({unique_drug_count} drugs):,\n")
    print(unique_drug_names)
    print()

In [ ]:
# @title Visualaization for Each Cluster Sales Over Monthly
# Function to plot all drugs' sales data within a cluster
def plot_cluster_sales(cluster_data, cluster_number):
    plt.figure(figsize=(15, 10))
    for drug in cluster_data.columns:
        drug_data = cluster_data[drug].dropna()
        if not drug_data.empty:
            plt.plot(drug_data.index, drug_data.values, marker='o', label=drug)
    plt.xlabel('Week')
    plt.ylabel('Quantity Sold')
    plt.title(f'Sales in Cluster {cluster_number}')
    # Customize x-axis ticks to show labels at even intervals of weeks
    num_weeks = len(cluster_data.index)
    xticks_loc = np.arange(0, num_weeks, 3)  # Show labels every 2 weeks
    xticks_labels = [cluster_data.index[i] for i in xticks_loc]
    plt.xticks(xticks_loc, xticks_labels, rotation=45)
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# Filter and transpose data by cluster
for cluster_number in sorted_data['Cluster'].unique():
    cluster_data = sorted_data[sorted_data['Cluster'] == cluster_number].drop(columns='Cluster').T
    plot_cluster_sales(cluster_data, cluster_number)


### Visualaization of sales for each drug in each cluster (Optional)

In [ ]:
# Plot for Cluster 0
#plot_drug_sales(cluster_0, 0)

In [ ]:
# Plot for Cluster 1
#plot_drug_sales(cluster_1, 1)

In [ ]:
# Plot for Cluster 2
#plot_drug_sales(cluster_2, 2)


# **Model Building**

In [ ]:
# @title SARIMA Model ON Clusters
# Forecast for each cluster
for cluster in range(optimal_clusters):
    cluster_data = filtered_data[filtered_data['Cluster'] == cluster]

    # Find the most frequently sold drug in the cluster
    top_drug = cluster_data.groupby('DrugName')['Quantity'].sum().nlargest(1).reset_index()['DrugName'].values[0]
    drug_data = cluster_data[cluster_data['DrugName'] == top_drug]

    # Set 'Monthly_Period' as the index
    drug_data.set_index('Months', inplace=True)

    # Split the data into training and testing sets
    train_size = int(len(drug_data) * 0.8)
    train_data, test_data = drug_data.iloc[:train_size], drug_data.iloc[train_size:]

    # Find the best SARIMA parameters
    best_mape = float("inf")
    best_pdq = None
    best_seasonal_pdq = None

    p = d = q = range(0, 3)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = SARIMAX(train_data['Quantity'],
                              order=param,
                              seasonal_order=param_seasonal,
                              enforce_stationarity=False,
                              enforce_invertibility=False)
                results = mod.fit(disp=False)
                forecast = results.get_forecast(steps=len(test_data))
                predicted_mean = forecast.predicted_mean
                mape = mean_absolute_percentage_error(test_data['Quantity'], predicted_mean)
                if mape < best_mape:
                    best_mape = mape
                    best_pdq = param
                    best_seasonal_pdq = param_seasonal
            except:
                continue

    print(f'Best SARIMA parameters for Cluster {cluster}: {best_pdq} x {best_seasonal_pdq}')
    print(f'Best MAPE for Cluster {cluster}: {best_mape}')

    # Fit the best SARIMA model
    best_sarima_model = SARIMAX(train_data['Quantity'],
                                order=best_pdq,
                                seasonal_order=best_seasonal_pdq,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    best_sarima_fit = best_sarima_model.fit(disp=False)

    # Forecast the future sales
    n_forecast = len(test_data)
    forecast = best_sarima_fit.get_forecast(steps=n_forecast)
    forecast_index = test_data.index

    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(train_data.index, train_data['Quantity'], label='Train')
    plt.plot(test_data.index, test_data['Quantity'], label='Test')
    plt.plot(forecast_index, forecast.predicted_mean, label='Forecast')
    plt.fill_between(forecast_index, forecast.conf_int()['lower Quantity'], forecast.conf_int()['upper Quantity'], color='k', alpha=0.1)
    plt.xlabel('Week')
    plt.ylabel('Quantity Sold')
    plt.title(f'SARIMA Forecast for Cluster {cluster} ({top_drug})')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# @title ## Forescasting Model for Top 20 individual Drug
# Create Dictionary to map month names into numeric values
dict_month = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
              'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

# Pivot the DataFrame Based on DrugName
pivoted_table = filtered_data.pivot_table(index='Months', columns='DrugName', values='Quantity', aggfunc='sum')

# Fill NaN values with 0
pivoted_table = pivoted_table.fillna(0)

# Sum the total quantity for each drug
total_quantity_per_drug = pivoted_table.sum()

# Select the top 20 drugs by total quantity
top_20_drugs = total_quantity_per_drug.nlargest(20).index

# Filter the pivoted table to include only the top 20 drugs
pivoted_table_top_20 = pivoted_table[top_20_drugs]

# Map month names to numeric values and sort the DataFrame
pivoted_table_top_20['MonthNum'] = pivoted_table_top_20.index.map(dict_month)
pivoted_table_top_20 = pivoted_table_top_20.sort_values('MonthNum')
pivoted_table_top_20.drop(['MonthNum'], axis=1, inplace=True)

# Forecast for each drug
for drug in pivoted_table_top_20.columns:
    drug_data = pivoted_table_top_20[[drug]].copy()

    # Split the data into training and testing sets
    train_size = int(len(drug_data) * 0.8)
    train_data, test_data = drug_data.iloc[:train_size], drug_data.iloc[train_size:]

    # Find the best SARIMA parameters
    best_mape = float("inf")
    best_pdq = None
    best_seasonal_pdq = None

    p = d = q = range(0, 3)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = SARIMAX(train_data[drug],
                              order=param,
                              seasonal_order=param_seasonal,
                              enforce_stationarity=True,
                              enforce_invertibility=False)
                results = mod.fit(disp=False)
                forecast = results.get_forecast(steps=len(test_data))
                predicted_mean = forecast.predicted_mean
                mape = mean_absolute_percentage_error(test_data[drug], predicted_mean)
                if mape < best_mape:
                    best_mape = mape
                    best_pdq = param
                    best_seasonal_pdq = param_seasonal
            except:
                continue

    print(f'Best SARIMA parameters for {drug}: {best_pdq} x {best_seasonal_pdq}')
    print(f'Best MAPE for {drug}: {best_mape}')

    # Fit the best SARIMA model
    best_sarima_model = SARIMAX(train_data[drug],
                                order=best_pdq,
                                seasonal_order=best_seasonal_pdq,
                                enforce_stationarity=True,
                                enforce_invertibility=False)
    best_sarima_fit = best_sarima_model.fit(disp=False)

    # Forecast the future sales
    n_forecast = len(test_data)
    forecast = best_sarima_fit.get_forecast(steps=n_forecast)
    forecast_index = test_data.index

    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(train_data.index, train_data[drug], label='Train')
    plt.plot(test_data.index, test_data[drug], label='Test')
    plt.plot(forecast_index, forecast.predicted_mean, label='Forecast')
    plt.fill_between(forecast_index, forecast.conf_int()['lower ' + drug], forecast.conf_int()['upper ' + drug], color='k', alpha=0.1)
    plt.xlabel('Month')
    plt.ylabel('Quantity Sold')
    plt.title(f'SARIMA Forecast for {drug}')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# @title Forcasting for next four months of 2023
import pandas as pd
import itertools
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt

# Create Dictionary to map month names into numeric values
dict_month = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
              'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

# Pivot the DataFrame Based on DrugName
pivoted_table = filtered_data.pivot_table(index='Months', columns='DrugName', values='Quantity', aggfunc='sum')

# Fill NaN values with 0
pivoted_table = pivoted_table.fillna(0)

# Sum the total quantity for each drug
total_quantity_per_drug = pivoted_table.sum()

# Select the top 20 drugs by total quantity
top_20_drugs = total_quantity_per_drug.nlargest(20).index

# Filter the pivoted table to include only the top 20 drugs
pivoted_table_top_20 = pivoted_table[top_20_drugs]

# Map month names to numeric values and sort the DataFrame
pivoted_table_top_20['MonthNum'] = pivoted_table_top_20.index.map(dict_month)
pivoted_table_top_20 = pivoted_table_top_20.sort_values('MonthNum')
pivoted_table_top_20.drop(['MonthNum'], axis=1, inplace=True)

# Predefined SARIMA parameters
sarima_params_dict = {
    0: {'pdq': (1, 1, 0), 'seasonal_pdq': (0, 0, 0, 12)},
    1: {'pdq': (2, 1, 0), 'seasonal_pdq': (0, 0, 0, 12)},
    2: {'pdq': (0, 1, 0), 'seasonal_pdq': (0, 0, 0, 12)},
    3: {'pdq': (1, 1, 0), 'seasonal_pdq': (0, 0, 0, 12)},
}

# Forecast for each drug
for i, drug in enumerate(pivoted_table_top_20.columns):
    drug_data = pivoted_table_top_20[[drug]].copy()

    # Check stationarity and apply differencing if necessary
    def check_stationarity(timeseries):
        from statsmodels.tsa.stattools import adfuller
        result = adfuller(timeseries)
        return result[1]  # p-value

    p_value = check_stationarity(drug_data[drug])
    if p_value > 0.05:
        drug_data[drug] = drug_data[drug].diff().dropna()

    # Get the predefined SARIMA parameters
    params = sarima_params_dict[i % len(sarima_params_dict)]
    pdq = params['pdq']
    seasonal_pdq = params['seasonal_pdq']

    # Fit the SARIMA model with predefined parameters
    best_sarima_model = SARIMAX(drug_data[drug],
                                order=pdq,
                                seasonal_order=seasonal_pdq,
                                enforce_stationarity=True,
                                enforce_invertibility=False)
    best_sarima_fit = best_sarima_model.fit(disp=False)

    # Forecast the future sales for Jan and Feb 2023
    n_forecast = 4  # Forecast for two additional months
    forecast = best_sarima_fit.get_forecast(steps=n_forecast)

    # Get the last date in the original data
    last_date = pd.to_datetime(drug_data.index[-1], format='%b')

    # Create forecast index for the next two months
    forecast_index = [last_date + pd.DateOffset(months=i) for i in range(1, n_forecast + 1)]

    # Convert forecast_index to month names
    forecast_index = [date.strftime('%b') for date in forecast_index]

    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(drug_data.index, drug_data[drug], label='Actual')
    plt.plot(forecast_index, forecast.predicted_mean, label='Forecast', marker='o')
    plt.fill_between(forecast_index, forecast.conf_int().iloc[:, 0], forecast.conf_int().iloc[:, 1], color='k', alpha=0.1)
    plt.xlabel('Month')
    plt.ylabel('Quantity Sold')
    plt.title(f'SARIMA Forecast for {drug} (Including Jan and Feb 2023)')
    plt.legend()
    plt.grid(True)
    plt.show()

import joblib, pickle

# Save essential components
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('sarima_params_dict.pkl', 'wb') as f:
    pickle.dump(sarima_params_dict, f)

with open('filtered_data.pkl', 'wb') as f:
    pickle.dump(filtered_data, f)

In [ ]:
import joblib, pickle

# Save essential components
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('sarima_params_dict.pkl', 'wb') as f:
    pickle.dump(sarima_params_dict, f)

with open('filtered_data.pkl', 'wb') as f:
    pickle.dump(filtered_data, f)

In [ ]:
import os
os.chdir('/Users/pavankumar/Projects/Medical Inventory Management')
os.getcwd  ()

## LSTM Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import backend as K

# Assuming filtered_data DataFrame is defined
# List of unique drug names
top_drug_names = filtered_data['DrugName'].unique()

# Convert DrugName to categorical and encode it
filtered_data['DrugName'] = filtered_data['DrugName'].astype('category')
filtered_data['DrugName_cat'] = filtered_data['DrugName'].cat.codes

# Scale the Quantity feature
scaler = MinMaxScaler(feature_range=(0, 1))
filtered_data['Quantity_scaled'] = scaler.fit_transform(filtered_data['Quantity'].values.reshape(-1, 1))

# Prepare the data for LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Define the number of time steps
TIME_STEPS = 10

# Create the LSTM input sequences and target values
X, y = create_dataset(filtered_data[['Quantity_scaled', 'Cluster', 'DrugName_cat']], filtered_data['Quantity_scaled'], TIME_STEPS)

# Split the data into training and testing sets
split_index = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split_index], X[split_index:], y[:split_index], y[split_index:]

# Define MAPE custom metric
def mape_metric(y_true, y_pred):
    epsilon = 1e-10  # small value to avoid division by zero
    return K.mean(K.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# Define the custom MAPE metric for Keras
def mape_metric(y_true, y_pred):
    epsilon = 1e-10  # small value to avoid division by zero
    mask = K.greater(y_true, epsilon)  # create a mask for non-zero true values
    masked_true = K.cast(mask, K.floatx()) * y_true  # apply mask to true values
    masked_pred = K.cast(mask, K.floatx()) * y_pred  # apply mask to predicted values
    return K.mean(K.abs((masked_true - masked_pred) / (masked_true + epsilon))) * 100

# Define the LSTM model architecture
model = Sequential([
    LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[mape_metric])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate the model
mse, mape = model.evaluate(X_test, y_test, verbose=0)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')


# **Deployment Part Essentials**

In [ ]:
data = pd.read_excel('/Users/pavankumar/Projects/Medical Inventory Management/Datasets/Medical Inventory Optimaization Dataset.xlsx')


In [ ]:
data.drop_duplicates(inplace = True)
data.reset_index(drop = True, inplace = True)
group_cols = ['Typeofsales','Specialisation','Dept']

for col in ['Formulation', 'DrugName', 'SubCat', 'SubCat1']:
    data[col] = data.groupby(group_cols)[col].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

data.reset_index(drop = True, inplace = True)
data.dropna(inplace = True)
data['Dateofbill'] = pd.to_datetime(data['Dateofbill'])
data = data.sort_values(by = 'Dateofbill', ascending = True)
data['Months'] = data['Dateofbill'].dt.strftime("%b")

Monthly_data = data.groupby(['DrugName', 'Months'])['Quantity'].sum().reset_index()
# Create Dictionary to map month names into numeric values
dict_month = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
              'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
# Map month names to numeric values and add a new column 'MonthNum'
Monthly_data['MonthNum'] = Monthly_data['Months'].map(dict_month)
# Sort the DataFrame by 'MonthNum'
Monthly_data = Monthly_data.sort_values(by='MonthNum')
# Drop the 'Months' column if it's no longer needed
Monthly_data.drop(['MonthNum'], axis=1, inplace=True)
# Display the result
display(Monthly_data)

top_drugs = data.groupby(['DrugName'], as_index = False)['Quantity'].sum()
top_drugs.sort_values(by = 'Quantity',ascending= False,inplace = True)

top_num_drugs = top_drugs.head(50)
display(top_num_drugs)

#Filter the original dataset to include only the top 20 drugs
top_drug_names = top_num_drugs['DrugName'].tolist()
filtered_data = Monthly_data[Monthly_data['DrugName'].isin(top_drug_names)]
filtered_data